# Graph-level inputs and outputs

## Introduction

When constructing complex workflows, you may encounter tasks that share input parameters (e.g. `code`). Also, you may wish to reorder or rename internal task outputs at the top level (e.g. `wg.outputs.optimized_stuff` instead of `wg.outputs.optimize.stuff`).

`WorkGraph` allows you to define **graph-level inputs and outputs** to:

- **Share inputs** across multiple tasks in a graph.
- **Aggregate outputs** from internal tasks, optionally organizing or renaming them.
- **Simplify the interface** by exposing shared parameters to users (retaining the flexibility of internal task parameter definitions).

## Defining graph-level inputs and outputs

In [1]:
from aiida_workgraph import WorkGraph, task

with WorkGraph("GraphLevelInputsSimple") as wg:
    wg.inputs = dict.fromkeys(["x", "y", "z"])  # define graph-level inputs
    outputs1 = wg.inputs.x + wg.inputs.y
    wg.outputs.sum = outputs1  # define graph-level output for the sum
    outputs2 = outputs1 * wg.inputs.z
    wg.outputs.product = outputs2  # define graph-level output for the product


wg.to_html()

### Run the workgraph


In [2]:
from aiida import load_profile

_ = load_profile()

wg.submit(
    {
        "graph_inputs": {
            "x": 1,
            "y": 2,
            "z": 3,
        },
    },
    wait=True,
)

print("Graph-level outputs:")
print("Sum:", wg.outputs.sum.value)
print("Product:", wg.outputs.product.value)

WorkGraph process created, PK: 10661
Process 10661 finished with state: FINISHED
Graph-level outputs:
Sum: uuid: 4ab741fa-788e-4d2c-9fad-07dd04997365 (pk: 10665) value: 3
Product: uuid: 2f97974e-4689-4ff9-8d76-d1c8d71f0220 (pk: 10669) value: 9


## Providing graph-level inputs metadata

Graph-level inputs can also be defined using `wg.add_input(...)`. The method allows you to provide an identifier (e.g. `workgraph.any`) to the input, which is used for validation and for clearer identification when visualizing the graph.

In [3]:
with WorkGraph("GraphLevelInputsMetadata") as wg:
    wg.add_input("workgraph.int", "x")  # validated as an integer
    wg.add_input("workgraph.int", "y")
    wg.add_input("workgraph.int", "z")
    wg.outputs.result = (wg.inputs.x + wg.inputs.y) * wg.inputs.z

When validating an input with a type identifier, it also associates the input port with a GUI component, allowing users to interact with the input in a more user-friendly way. For example, if you define an input as `workgraph.int`, it will be presented as an integer input field in the GUI.

In the future, further details may be added when defining inputs, e.g., default values, descriptions, help messages, etc.

## Nested graph-level inputs and outputs

Graph-level inputs and outputs can be **nested** allowing you to group related parameters and results. Here we're using the `add_task` interface to more clearly define the names of our tasks.

In [4]:
@task
def add(x, y):
    return x + y


@task
def multiply(x, y):
    return x * y


with WorkGraph("GraphLevelInputsNested") as wg:
    wg.inputs = {
        "add": {
            "first": dict.fromkeys(["x", "y"]),
            "second": dict.fromkeys(["x", "y"]),
        },
        "multiply": dict.fromkeys(["y"]),
    }
    first_add = wg.add_task(
        add,
        name="first_add",
        x=wg.inputs.add.first.x,
        y=wg.inputs.add.first.y,
    )
    second_add = wg.add_task(
        add,
        name="second_add",
        x=wg.inputs.add.second.x,
        y=wg.inputs.add.second.y,
    )
    third_add = wg.add_task(
        add,
        name="third_add",
        x=first_add.outputs.result,
        y=second_add.outputs.result,
    )
    multiply_task = wg.add_task(
        multiply,
        name="multiply",
        x=third_add.outputs.result,
        y=wg.inputs.multiply.y,
    )
    wg.outputs.results = {
        "sums": {
            "first": first_add.outputs.result,
            "second": second_add.outputs.result,
            "third": third_add.outputs.result,
        },
        "product": multiply_task.outputs.result,
    }


wg.to_html()

### Run the workgraph

We can now run our workgraph with a clear input layout. Note that `WorkGraph` will automatically serialize the raw Python data into the corresponding AiiDA Data nodes (e.g., an `int` becomes `orm.Int`, a `str` becomes `orm.Str`, etc.) before execution. The exact serialization logic and all supported types (and how to register your own custom serializers) are described in detail in the **Data Serialization** section.

In [5]:
wg.submit(
    inputs={
        "graph_inputs": {
            "add": {
                "first": {
                    "x": 1,
                    "y": 2,
                },
                "second": {
                    "x": 3,
                    "y": 4,
                },
            },
            "multiply": {
                "y": 5,
            },
        }
    },
    wait=True,
)


print("Results:")
print("  Sums:")
print("    First:", wg.outputs.results.sums.first.value)
print("    Second:", wg.outputs.results.sums.second.value)
print("    Third:", wg.outputs.results.sums.third.value)
print("  Product:", wg.outputs.results.product.value)

WorkGraph process created, PK: 10675
Process 10675 finished with state: FINISHED
Results:
  Sums:
    First: uuid: 660c049d-209a-4bdc-8474-6627a37da70d (pk: 10679) value: 3
    Second: uuid: 478215e2-4a43-4168-8794-0ec402065afd (pk: 10683) value: 7
    Third: uuid: a4fb30cd-e94c-4dc2-8521-729020d0b00b (pk: 10687) value: 10
  Product: uuid: 44a4ee01-c440-4dbd-94be-29bbdeaec561 (pk: 10691) value: 50


When we inspect the outputs of the `WorkGraph` process, we see `sums` and `product` are grouped under the `results` output.

In [6]:
!verdi process show {wg.pk}

Property     Value
-----------  ------------------------------------
type         WorkGraph<GraphLevelInputsNested>
state        Finished [0]
pk           10675
uuid         49569334-7906-46d8-8ba5-e6f51f0081a8
label        GraphLevelInputsNested
description
ctime        2025-07-13 05:58:01.663638+00:00
mtime        2025-07-13 05:58:04.381322+00:00

Inputs                                         PK     Type
---------------------------------------------  -----  ------
workgraph_data
    meta_sockets
        graph_inputs
            sockets
                add
                    sockets
                        first
                            sockets
                                x
                                    property
                                        value  10670  Int
                                y
                                    property
                                        value  10671  Int
                        second
                            sockets


## Summary

In this section, you learned how to:

- Use `wg.inputs = {...}` to define many inputs at once, or to define nested (namespaced) inputs to group related parameters
- Use `wg.add_input(...)` to define a graph-level input and provide additional metadata (e.g., type validation)
- Use graph-level inputs in tasks (`wg.inputs.<name>`)
- Use `wg.outputs.<name>` to expose graph-level outputs from internal tasks